## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi</a>  2024/10/10

### 这是<a href="https://www.datafountain.cn/competitions/1045">基于运营商文本数据的知识库检索</a>比赛的baseline.这里将使用Kaggle平台和<a href="https://github.com/gomate-community/GoMate">GoMate</a>框架来完成。

### GoMate框架做RAG任务可以参考<a href="https://www.kaggle.com/code/yunsuxiaozi/gomate-framework-usage">Gomate framework usage</a>。



# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">1.将PDF文件解析为text文本数据</h1></span>

先从PDF中读取出文本数据,然后将文本中无法用utf-8 encode的文本去掉.

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [2]:
import PyPDF2
import os
from tqdm import tqdm
document_path="/kaggle/input/2024ccf-retrieval/retrieval/A_document/"
pdfs=sorted(os.listdir(document_path))
texts=''
for i in tqdm(range(len(pdfs))):
    with open(f'{document_path}{pdfs[i]}', 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        # 读取每一页
        for page_num in range(len(reader.pages)):
            # 提取文本
            page_obj = reader.pages[page_num]
            text = page_obj.extract_text()
            texts+=text
    texts+='\n'
print(f"len(texts):{len(texts)}")
final_texts=''
def is_utf8(uchar):
    try:
        uchar.encode('utf-8')
        return True
    except:
        return False
for i in range(len(texts)):
    if is_utf8(texts[i]):
        final_texts+=texts[i]        
print(f"len(final_texts):{len(final_texts)}")

100%|██████████| 120/120 [00:59<00:00,  2.02it/s]


len(texts):847111
len(final_texts):846468


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">2.GoMate框架的配置</h1></span>

In [3]:
!git clone https://github.com/gomate-community/GoMate.git #gomate是配置化模块化的RAG框架,可靠的输入和可信的输出

Cloning into 'GoMate'...
remote: Enumerating objects: 1431, done.
remote: Counting objects: 100% (558/558), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 1431 (delta 347), reused 462 (delta 276), pack-reused 873 (from 1)
Receiving objects: 100% (1431/1431), 55.92 MiB | 33.50 MiB/s, done.
Resolving deltas: 100% (834/834), done.


In [4]:
cd /kaggle/working/GoMate

/kaggle/working/GoMate


In [5]:
!pip install -q -e .  #安装依赖

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">3.文本的分块</h1></span>

这里简单粗暴的去除一些停用词,然后根据句号、感叹号和问号划分文本,最后去除无用的语句

In [6]:
def generate_corpus(text,chunk_size,stopwords=['了','的','年','月']):
    #去停用词
    for word in stopwords:
        text=text.replace(word,'')   
    corpus=[]
    cur_text=""
    for i in range(len(text)):
        if ((text[i] in ['。','？','！']) and (len(cur_text)>=chunk_size)) or(i==len(text)-1):
            corpus.append(cur_text+text[i])
            cur_text=''
        else:
            cur_text+=text[i]
    return corpus
corpus=generate_corpus(final_texts,chunk_size=64)

corpus=[text for text in corpus if '本文档为2024CCFBDCI比赛用语料的一部分' not in text]

print(len(corpus))  
for i in range(len(corpus)-1):
    corpus[i]=(corpus[i]+corpus[i+1])

6178


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">4.检索器设置</h1></span>

这里使用BM25检索器+稠密向量检索器

In [7]:
from gomate.modules.retrieval.bm25s_retriever import BM25RetrieverConfig#基于BM25的检索器
from gomate.modules.retrieval.dense_retriever import DenseRetrieverConfig#基于稠密向量表示的检索器
from gomate.modules.retrieval.hybrid_retriever import HybridRetriever, HybridRetrieverConfig#实现混合检索器的类


bm25_config = BM25RetrieverConfig(
    method='lucene',
    index_path='indexs/description_bm25.index',
    k1=1.6,
    b=0.7
)
bm25_config.validate()

dense_config = DenseRetrieverConfig(
    model_name_or_path="BAAI/bge-large-zh-v1.5",
    dim=1024,
    index_path='indexs/dense_cache'
)
config_info = dense_config.log_config()


# 由于分数框架不在同一维度，建议可以合并
hybrid_config = HybridRetrieverConfig(
    bm25_config=bm25_config,
    dense_config=dense_config,
    bm25_weight=0.7,  # bm25检索结果权重
    dense_weight=0.3  # dense检索结果权重
)
hybrid_retriever = HybridRetriever(config=hybrid_config)
#构建索引
hybrid_retriever.build_from_texts(corpus)
#保存索引
hybrid_retriever.save_index()
# 加载索引
hybrid_retriever.load_index()

RAG Vocab: /kaggle/working/GoMate/gomate/modules/document/huqie.txt
RAG Trie: /kaggle/working/GoMate/gomate/modules/document/huqie.txt.trie
BM25 configuration is valid.


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

BM25S Count Tokens:   0%|          | 0/6178 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/6178 [00:00<?, ?it/s]

Building index: 100%|██████████| 194/194 [03:50<00:00,  1.19s/it]


Finding newlines for mmindex:   0%|          | 0.00/4.45M [00:00<?, ?B/s]

Index saving to：indexs/dense_cache
Index saved successfully to indexs/dense_cache
Index loaded successfully from indexs/dense_cache


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">5.问题的检索和重排</h1></span>

这里换了一个更好的rerank模型:BAAI/bge-reranker-v2-M3,将答案截取前450个字,因为目前测试下来这样最好。

In [8]:
import pandas as pd
submission=pd.read_csv("/kaggle/input/2024ccf-retrieval/retrieval/submit_example.csv")


from gomate.modules.reranker.bge_reranker import BgeRerankerConfig, BgeReranker#基于bert的重排器
reranker_config = BgeRerankerConfig(
    model_name_or_path="BAAI/bge-reranker-v2-M3"#'BAAI/bge-reranker-large'
)
bge_reranker = BgeReranker(reranker_config)

question=list(submission['question'].values)

answer=[]
topk=25
for q in question:

    search_docs = hybrid_retriever.retrieve(q,top_k=topk)
    search_docs = bge_reranker.rerank(
        query=q,
        documents=[doc['text'] for idx, doc in enumerate(search_docs)]
    )
    #截断防止超长度
    answer.append( ''.join([doc['text'] for idx, doc in enumerate(search_docs)][:3]) )
print(f"answer[0]:{answer[0]}")
answer=[a[:450] for a in answer]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

answer[0]:
我们坚定践行网络强国、数字中国、智慧社会战略部署，今天中国联通，正在从传统运营商加
速向数字科技领军企业转变，实现四个维度转型升级：一是联接规模和联接结构升维，从过去
连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为
主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以
5G、云计算、大数据、人工智能、区块链为代表新一代信息技术和实体经济结合，服务数字政府、
数字社会、数字经济能力不断增强；四是发展理念升维，我们以传统市场驱动为主转变为市场驱
动和创新驱动相结合发展模式，尤其是加大科技创新及人才方面投入力度，创新发展动能得
到空前释放。
面对数字经济巨大机遇和广阔蓝海，我们积极储备核心能力，全资本支出达到人民币742亿
元，企业行稳致远根本得到巩固夯实。公司延续近来强劲稳健财务状况，资产负债率维持在
46%合理水平，财务费用同比大幅下降873%，财务实力和抗风险能力不断增强。
（一）整体业绩
2022，公司实现经营发展“四个新高”：一是实现营业收入3,549亿元，同比增长达到
8.3%，增速创近9新高；二是实现利润总额204亿元，归属于母公司净利润73亿元，同比增长达
到15.8%，在剔除非经营性损益后1，净利润规模创公司上市以来新高；三是产业互联网收入占主营业
务收入比首次突破20%，创新业务收入占比达到历史新高；四是EBITDA达到990亿元，创公司上市以
来新高。
我们坚定践行网络强国、数字中国、智慧社会战略部署，今天中国联通，正在从传统运营商加
速向数字科技领军企业转变，实现四个维度转型升级：一是联接规模和联接结构升维，从过去
连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为
主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以
5G、云计算、大数据、人工智能、区块链为代表新一代信息技术和实体经济结合，服务数字政府、
数字社会、数字经济能力不断增强；四是发展理念升维，我们以传统市场驱动为主转变为市场驱
动和创新驱动相结合发展模式，尤其是加大科技创新及人才方面投入力度，创新发展动能得
到空前释放。到2025，我
国数字经济规模将超越60万亿3，庞大市场、发展蓝海、炫目技术，都为我们创造极其广阔

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">6.提交文件</h1></span>

In [9]:
from transformers import AutoTokenizer, AutoModel#自动加载分词器和自动加载预训练模型
import pandas as pd#读取表格数据的库
import numpy as np
import torch#深度学习框架pytorch
from tqdm import tqdm
print("load embedding tokenizer")
#允许远程加载模型,模型加载的时候将模型转成半精度浮点数并移动到GPU上。
embed_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-zh-v1.5", trust_remote_code=True)
print("load embedding model")
embed_model=AutoModel.from_pretrained("BAAI/bge-large-zh-v1.5",trust_remote_code=True)
#模型转换成评估模式
embed_model.eval()
def embedding(texts,batch_size=5):
    embed_vector=[]
    for i in tqdm(range(0,len(texts),batch_size)):
        text_input=embed_tokenizer(texts[i:i+batch_size], max_length=1024, padding=True, truncation=True,
                                               return_tensors='pt')
        with torch.no_grad():
            model_output = embed_model(**text_input)
        #,截取开头的cls,据说cls有了全局的特征,所以[:, 0]就是cls pool.
        batch_embeddings=model_output['last_hidden_state'][:, 0]
        #(batch_size,1024)
        batch_embeddings=torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
        embed_vector.append(batch_embeddings.numpy())
    return np.vstack(embed_vector)

submission['answer']=answer
answer_vector=embedding(answer)
answer_embedding=[ ','.join([str(a) for a in answer_vector[i]]) for i in range(len(answer_vector))]
submission['embedding']=answer_embedding
submission.to_csv("1010gomate_rerankv2m3_baseline.csv",index=None)
submission.head()

load embedding tokenizer
load embedding model


100%|██████████| 20/20 [02:12<00:00,  6.64s/it]


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,\n我们坚定践行网络强国、数字中国、智慧社会战略部署，今天中国联通，正在从传统运营商加\n速...,"-0.033276223,-0.003946108,-0.025320176,0.00125..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,\n创新业务再提速\n公司产业互联网继续按下快进键，2022收入首破700亿大关，同比增长达...,"-0.024949672,-0.01253751,-0.023683239,0.000173..."
2,3,根据2022年度报告，中国联通的企业定位是什么？,中国联合网络通信股份有限公司 ...,"-0.042192217,-0.02312958,-0.046016775,0.010921..."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,中国联合网络通信股份有限公司 ...,"-0.042192217,-0.02312958,-0.046016775,0.010921..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,\n适度加大战略投入，基础网络能力大幅提升 \n中国联通始终坚持网络在企业发展中基础地位...,"-0.008452993,-0.011579074,-0.041638438,-0.0010..."
